In [1]:
import torchio as tio
import os
import nibabel as nib
import numpy as np
import torch

import torchio as tio
from nibabel.affines import voxel_sizes
from scipy import ndimage

In [2]:
print("## Simple define")
# define variable
#gt_folder = './DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/seg_gt/test'
#ct_folder = './DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/unetr_pp_Data_plans_v2.1_stage1/test'
#gt_folder = './DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/seg_gt/3Dircard'
#ct_folder = './DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/unetr_pp_Data_plans_v2.1_stage1/3Dircard'
#folder = './output_synapse/3d_fullres/Task002_Synapse/unetr_pp_trainer_synapse__unetr_pp_Plansv2.1/fold_4/validation_raw'
#output_folder = './output_synapse/example/couinaud'
#name = os.listdir(folder)
#name = [nii for nii in name if '.nii' in nii]
#name = [nii for nii in name if '.nii.gz' in nii]
#print(name)

#tmp_lst=['volume-15.nii', 'volume-18.nii', 'volume-28.nii', 'volume-3.nii', 'volume-33.nii', 'volume-37.nii', 'volume-42.nii', 'volume-47.nii', 'volume-5.nii', 'volume-54.nii', 'volume-70.nii', 'volume-73.nii', 'volume-80.nii']
#tmp_lst=['volume-15.nii', 'volume-18.nii', 'volume-28.nii', 'volume-3.nii', 'volume-33.nii', 'volume-37.nii', 'volume-42.nii', 'volume-47.nii', 'volume-5.nii', 'volume-54.nii', 'volume-70.nii', 'volume-73.nii', 'volume-80.nii', 'volume-17.nii', 'volume-43.nii', 'volume-77.nii' ]

## Simple define


In [3]:
def remove_all_but_the_largest_connected_component(image: np.ndarray, for_which_classes: list, volume_per_voxel: float,
                                                   minimum_valid_object_size: dict = None):
    """
    removes all but the largest connected component, individually for each class
    :param image:
    :param for_which_classes: can be None. Should be list of int. Can also be something like [(1, 2), 2, 4].
    Here (1, 2) will be treated as a joint region, not individual classes (example LiTS here we can use (1, 2)
    to use all foreground classes together)
    :param minimum_valid_object_size: Only objects larger than minimum_valid_object_size will be removed. Keys in
    minimum_valid_object_size must match entries in for_which_classes
    :return:
    """
    if for_which_classes is None:
        for_which_classes = np.unique(image)
        for_which_classes = for_which_classes[for_which_classes > 0]

    assert 0 not in for_which_classes, "cannot remove background"
    largest_removed = {}
    kept_size = {}
    for c in for_which_classes:
        if isinstance(c, (list, tuple)):
            c = tuple(c)  # otherwise it cant be used as key in the dict
            mask = np.zeros_like(image, dtype=bool)
            for cl in c:
                mask[image == cl] = True
        else:
            mask = image == c
        # get labelmap and number of objects
        lmap, num_objects = ndimage.label(mask.astype(int))

        # collect object sizes
        object_sizes = {}
        for object_id in range(1, num_objects + 1):
            object_sizes[object_id] = (lmap == object_id).sum() * volume_per_voxel

        largest_removed[c] = None
        kept_size[c] = None

        if num_objects > 0:
            # we always keep the largest object. We could also consider removing the largest object if it is smaller
            # than minimum_valid_object_size in the future but we don't do that now.
            maximum_size = max(object_sizes.values())
            kept_size[c] = maximum_size

            for object_id in range(1, num_objects + 1):
                # we only remove objects that are not the largest
                if object_sizes[object_id] != maximum_size:
                    # we only remove objects that are smaller than minimum_valid_object_size
                    remove = True
                    if minimum_valid_object_size is not None:
                        remove = object_sizes[object_id] < minimum_valid_object_size[c]
                    if remove:
                        image[(lmap == object_id) & mask] = 0
                        if largest_removed[c] is None:
                            largest_removed[c] = object_sizes[object_id]
                        else:
                            largest_removed[c] = max(largest_removed[c], object_sizes[object_id])
    return image, largest_removed, kept_size

In [28]:
def label_7_post_processing(class_edge, four_edge, pred_array, change_label):
    #### Find intersection edges
    intersection_edge = class_edge*four_edge

    #### Select line points
    y_coord, x_coord = np.where(intersection_edge == 1) # (y_list, x_list)
    
    if len(y_coord) == 0:
        pass
        
    else:
        indexing = np.where((y_coord >= min(y_coord)) & (y_coord <= min(y_coord) + 20))
        x_index = x_coord[indexing]
        y_index = y_coord[indexing]

        #intersection_edge[y_index, x_index] = 5
        #plt.imshow(intersection_edge)

        #### calculate equation
        coefficients = np.polyfit(x_index, y_index, 1)
        polynomial = np.poly1d(coefficients)

        #### change label
        class_y, class_x = np.where((pred_array != change_label) & (pred_array != 0))
        change_y_coord = []
        change_x_coord = []

        threshold_y = polynomial(class_x)
        #plt.plot(class_x, threshold_y)

        for i in range(len(class_y)):
            if min(x_index) <= class_x[i]: # 시작되는 x보다는 오른쪽에 위치
                if threshold_y[i] <= class_y[i]:
                    change_y_coord.append(class_y[i])
                    change_x_coord.append(class_x[i])

        #### CCL -> two more region, remove components
        class_five_y_range, _ = np.where(pred_array==change_label)
        y_max, y_min = max(class_five_y_range), min(class_five_y_range)
        y_range = np.arange(y_min, y_max)
        class_four = np.where(pred_array==4, 1, 0)
        labels, num_objects = ndimage.label(class_four >= 1)
        if num_objects >= 2:
            for c in range(1, num_objects+1):
                sub_image_y, sub_image_x = np.where(labels==c)
                intersection_y = np.intersect1d(y_range, sub_image_y)   

                # 4 class in 5 region.
                if len(intersection_y) == len(np.unique(sub_image_y)): 
                    change_y_coord.extend(list(sub_image_y))
                    change_x_coord.extend(list(sub_image_x))

        pred_array[change_y_coord, change_x_coord] = change_label # 4 -> 5
        
    return pred_array

In [29]:
# 4 -> 8
# 4 -> 5
def label_4_post_processing(class_edge, four_edge, pred_array, change_label):
    #### Find intersection edges
    intersection_edge = class_edge*four_edge

    #### Select line points
    y_coord, x_coord = np.where(intersection_edge == 1) # (y_list, x_list)
    
    if len(y_coord) == 0:
        pass
        
    else:
        indexing = np.where((y_coord >= min(y_coord)) & (y_coord <= min(y_coord) + 20))
        x_index = x_coord[indexing]
        y_index = y_coord[indexing]

        #intersection_edge[y_index, x_index] = 5
        #plt.imshow(intersection_edge)

        #### calculate equation
        coefficients = np.polyfit(x_index, y_index, 1)
        polynomial = np.poly1d(coefficients)

        #### change label
        #class_y, class_x = np.where((pred_array != change_label) & (pred_array != 0))
        class_y, class_x = np.where((pred_array == 4))
        change_y_coord = []
        change_x_coord = []

        threshold_y = polynomial(class_x)
        #plt.plot(class_x, threshold_y)

        for i in range(len(class_y)):
            if min(x_index) <= class_x[i]: # 시작되는 x보다는 오른쪽에 위치
                if threshold_y[i] <= class_y[i]:
                    change_y_coord.append(class_y[i])
                    change_x_coord.append(class_x[i])

        #### CCL -> two more region, remove components
        class_five_y_range, _ = np.where(pred_array==change_label)
        y_max, y_min = max(class_five_y_range), min(class_five_y_range)
        y_range = np.arange(y_min, y_max)
        class_four = np.where(pred_array==4, 1, 0)
        labels, num_objects = ndimage.label(class_four >= 1)
        if num_objects >= 2:
            for c in range(1, num_objects+1):
                sub_image_y, sub_image_x = np.where(labels==c)
                intersection_y = np.intersect1d(y_range, sub_image_y)   

                # 4 class in 5 region.
                if len(intersection_y) == len(np.unique(sub_image_y)): 
                    change_y_coord.extend(list(sub_image_y))
                    change_x_coord.extend(list(sub_image_x))

        pred_array[change_y_coord, change_x_coord] = change_label # 4 -> 5
        
    return pred_array

In [30]:
# 6 -> 5
def label_6_post_processing(class_edge, five_edge, pred_array, change_label):
    #### Find intersection edges
    intersection_edge = class_edge*five_edge

    #### Select line points
    y_coord, x_coord = np.where(intersection_edge == 1) # (y_list, x_list)
    
    if len(y_coord) == 0:
        pass
        
    else:
        indexing = np.where((y_coord >= min(y_coord)) & (y_coord <= min(y_coord) + 20))
        x_index = x_coord[indexing]
        y_index = y_coord[indexing]

        #intersection_edge[y_index, x_index] = 5
        #plt.imshow(intersection_edge)

        #### calculate equation
        coefficients = np.polyfit(x_index, y_index, 1)
        polynomial = np.poly1d(coefficients)

        #### change label
        #class_y, class_x = np.where((pred_array != change_label) & (pred_array != 0))
        class_y, class_x = np.where((pred_array == 6))
        change_y_coord = []
        change_x_coord = []

        threshold_y = polynomial(class_x)
        #plt.plot(class_x, threshold_y)

        for i in range(len(class_y)):
            if min(x_index) <= class_x[i]: # 시작되는 x보다는 오른쪽에 위치
                change_y_coord.append(class_y[i])
                change_x_coord.append(class_x[i])

        #### CCL -> two more region, remove components
        class_five_y_range, _ = np.where(pred_array==change_label)
        y_max, y_min = max(class_five_y_range), min(class_five_y_range)
        y_range = np.arange(y_min, y_max)
        class_six = np.where(pred_array==6, 1, 0)
        labels, num_objects = ndimage.label(class_six >= 1)
        if num_objects >= 2:
            for c in range(1, num_objects+1):
                sub_image_y, sub_image_x = np.where(labels==c)
                intersection_y = np.intersect1d(y_range, sub_image_y)   

                # 4 class in 5 region.
                if len(intersection_y) == len(np.unique(sub_image_y)): 
                    change_y_coord.extend(list(sub_image_y))
                    change_x_coord.extend(list(sub_image_x))

        pred_array[change_y_coord, change_x_coord] = change_label # 4 -> 5
        
    return pred_array

In [31]:
def default_postprocessing(processed):
    #### CCL algorithm
    processed, largest_removed, kept_size = remove_all_but_the_largest_connected_component(
        image=processed,
        for_which_classes=[1,2,3,4,5,6,7,8],
        volume_per_voxel=3,
        minimum_valid_object_size=None  # 이 인자는 생략하거나 None으로 설정할 수 있dma
        )
    
    #### post-processing
    for i in range(len(processed)):
        #processed[i,...] = binary_fill_holes(processed[i,...])
        
        # (5,8) 동시 존재
        if (5 in processed[i, ...]) and (8 in processed[i, ...]):
            if np.sum(processed[i, ...]==8) > np.sum(processed[i, ...]==5):
                processed[i, ...] = np.where(processed[i, ...]==5, 8, processed[i, ...]) 
                # 자동으로 6 존재 X -> 7로 변환
                processed[i, ...] = np.where(processed[i, ...]==6, 7, processed[i, ...]) 
            else:
                processed[i, ...] = np.where(processed[i, ...]==8, 5, processed[i, ...]) 
                # 자동으로 7 존재 X -> 6로 변환
                processed[i, ...] = np.where(processed[i, ...]==7, 6, processed[i, ...]) 
        
        # (6,7) 동시 존재 
        elif (6 in processed[i, ...]) and (7 in processed[i, ...]):
            if np.sum(processed[i, ...]==7) > np.sum(processed[i, ...]==6):
                processed[i, ...] = np.where(processed[i, ...]==6, 7, processed[i, ...]) 
                # 자동으로 5 존재 X -> 8로 변환
                processed[i, ...] = np.where(processed[i, ...]==5, 8, processed[i, ...])
            else:
                processed[i, ...] = np.where(processed[i, ...]==7, 6, processed[i, ...]) 
                # 자동으로 8 존재 X -> 5로 변환
                processed[i, ...] = np.where(processed[i, ...]==8, 5, processed[i, ...])
    
        # (2,3) 동시 존재 
        # 0.831989 (2 -> 3)
        if (2 in processed[i, ...]) and (3 in processed[i, ...]):
            #if np.sum(processed[i, ...]==2) > np.sum(processed[i, ...]==3):
            processed[i, ...] = np.where(processed[i, ...]==2, 3, processed[i, ...]) 
            #else:
            #    processed[i, ...] = np.where(processed[i, ...]==3, 2, processed[i, ...]) 
            
    return processed

In [32]:
from scipy.ndimage import binary_erosion, median_filter, \
                            binary_dilation, binary_fill_holes, binary_closing

def couinaud_postprocessing(pred_array):
    
    # Using slope-line to post-processing
    #### Case1 4 -> 5 or 4 -> 8 ()
    for i in range(pred_array.shape[-1]):
        five_edge = binary_dilation(np.where(pred_array[0,...,i]==5, 1, 0)) - np.where(pred_array[0,...,i]==5, 1, 0)
        four_edge = np.where(pred_array[0,...,i]==4, 1, 0) - binary_erosion(np.where(pred_array[0,...,i]==4, 1, 0))
        eight_edge = np.where(pred_array[0,...,i]==8, 1, 0) - binary_erosion(np.where(pred_array[0,...,i]==8, 1, 0))
        
        if five_edge.sum() == 0 or four_edge.sum() == 0:
            eight_edge = binary_dilation(np.where(pred_array[0,...,i]==8, 1, 0)) - np.where(pred_array[0,...,i]==8, 1, 0)
            
            if eight_edge.sum() == 0 or four_edge.sum() == 0:
                continue
            
            else:
                # label 4 -> 8
                pred_array[0, ..., i] = label_4_post_processing(eight_edge, four_edge, pred_array[0, ..., i], 8)
                
        elif eight_edge.sum() == 0 or four_edge.sum() == 0:
            five_edge = binary_dilation(np.where(pred_array[0,...,i]==5, 1, 0)) - np.where(pred_array[0,...,i]==5, 1, 0)
            
            if five_edge.sum() == 0 or four_edge.sum() == 0:
                continue
            
            else:
                # label 4 -> 5
                pred_array[0, ..., i] = label_4_post_processing(five_edge, four_edge, pred_array[0, ..., i], 5)
            
        #else:
        #    # label 4 -> 5
        #    print("????")
        #    pred_array[0, ..., i] = label_4_post_processing(five_edge, four_edge, pred_array[0, ..., i], 5)
    
    #### Case2 6 -> 5 (|)
    # 기준선을 기준으로, 오른쪽이 5, 왼쪽이 6
    for i in range(pred_array.shape[-1]):
        five_edge = binary_dilation(np.where(pred_array[0,...,i]==5, 1, 0)) - np.where(pred_array[0,...,i]==5, 1, 0)
        six_edge = np.where(pred_array[0,...,i]==6, 1, 0) - binary_erosion(np.where(pred_array[0,...,i]==6, 1, 0))
        
        if six_edge.sum() == 0 or five_edge.sum() == 0:
            pass
        
        else:
            pred_array[0, ..., i] = label_6_post_processing(five_edge, six_edge, pred_array[0, ..., i], 5)
            
    #### Case3 7 & 8
    # Remove outlier
    '''
    for i in range(pred_array.shape[-1]):
        
        ## 7 outlier -> 8
        class_seven = np.where(pred_array[0, ..., i]==7, 1, 0)
        labels, num_objects = ndimage.label(class_seven >= 1)

        if num_objects >= 2:
            max_x = 0
            change_count = 0
            for c in range(1, num_objects+1):
                sub_image_y, sub_image_x = np.where(labels==c) 
                maximal_x = max(sub_image_x)

                if maximal_x > max_x:
                    max_x = maximal_x
                    change_7y_coord = sub_image_y
                    change_7x_coord = sub_image_x

                    if change_count >= 1:
                        pred_array[0, change_7y_coord, change_7x_coord, i] = 8 # 7 -> 8
                    change_count += 1
                    
        ## 8 outlier -> 7
        class_eight = np.where(pred_array[0, ..., i]==8, 1, 0)
        labels, num_objects = ndimage.label(class_eight >= 1)

        if num_objects >= 2:
            min_x = 9999
            change_count = 0
            for c in range(1, num_objects+1):
                sub_image_y, sub_image_x = np.where(labels==c) 
                minimal_x = min(sub_image_x)

                if maximal_x < min_x:
                    min_x = minimal_x
                    change_8y_coord = sub_image_y
                    change_8x_coord = sub_image_x

                    if change_count >= 1:
                        pred_array[0, change_8y_coord, change_8x_coord, i] = 7 # 8 -> 7
                    change_count += 1
    '''
    return pred_array

In [33]:
print("## Simple Example")
'''
flag_post1 = True
flag_post2 = True

#for i in range(len(name)):
i=19
pred = nib.load(os.path.join(folder, name[i]))
pred_array = pred.get_fdata()

pred_array = np.array(pred_array[np.newaxis, ...], dtype='int')
if flag_post1:
    print("Pairwise - (5,6), (7,8) & (2,3)...")
    pred_array[0] = default_postprocessing(pred_array[0])
pred_array = pred_array.transpose(0,3,2,1) # (1, 328, 328, 235)

if flag_post2:
    print("Using slope-line, and changing labels...")
    pred_array = couinaud_postprocessing(pred_array)
'''

## Simple Example


'\nflag_post1 = True\nflag_post2 = True\n\n#for i in range(len(name)):\ni=19\npred = nib.load(os.path.join(folder, name[i]))\npred_array = pred.get_fdata()\n\npred_array = np.array(pred_array[np.newaxis, ...], dtype=\'int\')\nif flag_post1:\n    print("Pairwise - (5,6), (7,8) & (2,3)...")\n    pred_array[0] = default_postprocessing(pred_array[0])\npred_array = pred_array.transpose(0,3,2,1) # (1, 328, 328, 235)\n\nif flag_post2:\n    print("Using slope-line, and changing labels...")\n    pred_array = couinaud_postprocessing(pred_array)\n'

In [34]:
# define path (설정)
gt_folder = './DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/seg_gt/test'
pred_folder = './output_synapse/3d_fullres/Task002_Synapse/unetr_pp_trainer_synapse__unetr_pp_Plansv2.1/fold_4/validation_raw'
#pred_folder = './output_synapse/example/couinaud/'
# 256x256

# make nii files
pred_name = os.listdir(pred_folder)
pred_name = [nii for nii in pred_name if '.nii' in nii]
print(len(pred_name)) # LiTS: 20

3


In [35]:
from tqdm import tqdm
import pandas as pd

flag_post1 = True
flag_post2 = True

metric_df = pd.DataFrame(columns=['id', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5',\
                                      'label_6', 'label_7', 'label_8', 'label_all'])

for i in tqdm(range(len(pred_name))):
    #i=19 # simple example
    
    #########################
    #pred = tio.ScalarImage(os.path.join(pred_folder, pred_name[i]))
    pred = nib.load(os.path.join(pred_folder, pred_name[i]))
    #print(pred.shape)
    pred_array = pred.get_fdata()

    pred_array = np.array(pred_array[np.newaxis, ...], dtype='int')
    if flag_post1:
        print("Pairwise - (5,6), (7,8) & (2,3)...")
        pred_array[0] = default_postprocessing(pred_array[0])
    pred_array = pred_array.transpose(0,3,2,1) # (1, 328, 328, 235)

    if flag_post2:
        print("Using slope-line, and changing labels...")
        pred_array = couinaud_postprocessing(pred_array)

    #########################
    gt = tio.ScalarImage(os.path.join(gt_folder, pred_name[i]+'.gz')) 
    #print(gt.shape)

    couinaud_gt = np.zeros(pred_array[0].shape)
    gt_copy_data = gt.data.clone()
    unique_label = np.unique(gt.data)
    resample = tio.Resample()

    if len(unique_label) != 9:
        raise Exception('Bug data?')
    else:
        for k in unique_label:
            #print(k)
            if k != 0:
                #a = tio.ScalarImage(path+'hepaticvessel_020.nii.gz')
                gt.data = np.where(gt.data>=k, 1, 0)
                sub_gt = resample(gt)
                sub_gt = sub_gt.data[0]
                sub_gt = np.array(sub_gt)
                #sub_gt = np.where(sub_gt==1, k, 0)
                couinaud_gt += sub_gt
                gt.data = gt_copy_data

    if len(np.unique(couinaud_gt)) != 9: # error check # 0,1,2,3,4,5,6,7,8,9
        print(name)
        raise Exception("No label")

    #gt = np.transpose(couinaud_gt, (2,1,0)) # TODO: 9/24 axial
    #print(couinaud_gt.shape)
    
    #########################
    def cal_DC_and_DG(pred, gt):
        dice_list = []
        # inersection, union, dice 계산

        for i in range(1,9):
            pred_i = np.where(pred==i, 1, 0)
            gt_i = np.where(gt==i, 1, 0)
            intersection = (pred_i*gt_i).sum()
            union = pred_i.sum() + gt_i.sum()
            dice = 2*intersection/union

            dice_list.append(dice)

        dice = sum(dice_list)/len(dice_list)
        dice_list.append(dice)

        return dice_list[0], dice_list[1], dice_list[2], dice_list[3], dice_list[4], dice_list[5], \
                dice_list[6], dice_list[7], dice_list[8]

    # calculation metric
    one, two, three, four, five, six, seven, eight, every = cal_DC_and_DG(pred_array[0], couinaud_gt)
    print("Dice score:", every)
    metric_df.loc[i] = [pred_name[i], one, two, three, four, five, six, seven, eight, every]
    
    #if i in [4, 13, 19, 24] :
    #    nifti_file = nib.Nifti1Image(pred_array[0], np.eye(4))
    #    nifti_file.to_filename('sample_'+str(i)+'.nii.gz') 
        
    #    nifti_file = nib.Nifti1Image(couinaud_gt, np.eye(4))
    #    nifti_file.to_filename('coui_gt'+str(i)+'.nii.gz') 
    #break
    
    

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Pairwise - (5,6), (7,8) & (2,3)...
Using slope-line, and changing labels...


C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\

Dice score: 0.9035688578069109
Pairwise - (5,6), (7,8) & (2,3)...
Using slope-line, and changing labels...


C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\2722269027.py:21: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_index, y_index, 1)
C:\Users\user\AppData\

Dice score: 0.8976168810655278
Pairwise - (5,6), (7,8) & (2,3)...
Using slope-line, and changing labels...


C:\Users\user\AppData\Local\Temp\ipykernel_39708\4289698757.py:45: DeprecationWarning: Call to deprecated function (or staticmethod) data. (Setting the image data with the property setter is deprecated. Use the set_data() method instead) -- Deprecated since version 0.18.16.
  gt.data = np.where(gt.data>=k, 1, 0)
C:\Users\user\AppData\Local\Temp\ipykernel_39708\4289698757.py:51: DeprecationWarning: Call to deprecated function (or staticmethod) data. (Setting the image data with the property setter is deprecated. Use the set_data() method instead) -- Deprecated since version 0.18.16.
  gt.data = gt_copy_data
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:52<00:00, 37.41s/it]

Dice score: 0.8801670595325827


In [36]:
# show result
metric_df.mean(axis=0) # 0.8265 -> 0.8364

C:\Users\user\AppData\Local\Temp\ipykernel_39708\3172736742.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  metric_df.mean(axis=0) # 0.8265 -> 0.8364


label_1      0.866342
label_2      0.923298
label_3      0.915281
label_4      0.872273
label_5      0.887518
label_6      0.875869
label_7      0.917015
label_8      0.892678
label_all    0.893784
dtype: float64

In [37]:
metric_df.describe()

,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_all
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.866342,0.923298,0.915281,0.872273,0.887518,0.875869,0.917015,0.892678,0.893784
std,0.029563,0.025076,0.018841,0.043538,0.037128,0.048378,0.023922,0.031666,0.012163
min,0.835567,0.898529,0.899356,0.843506,0.847934,0.820326,0.892841,0.857456,0.880167
25%,0.852251,0.910611,0.904882,0.847228,0.870492,0.859406,0.905185,0.879620,0.888892
50%,0.868936,0.922693,0.910407,0.850951,0.893051,0.898486,0.917529,0.901783,0.897617
75%,0.881729,0.935682,0.923244,0.886657,0.907310,0.903641,0.929103,0.910288,0.900593
max,0.894523,0.948671,0.936080,0.922362,0.921570,0.908797,0.940676,0.918794,0.903569


In [13]:
# df save
metric_df.loc[-1] = metric_df.mean(axis=0)
metric_df.to_csv('./excel_result/Couidnaud_test50_nopost_final.csv', index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_30412\2807981540.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  metric_df.loc[-1] = metric_df.mean(axis=0)
